In [77]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix

# Charger les datasets
data_gender = pd.read_csv(r'C:\Users\Moi\Downloads\gender_submission.csv')
data_train = pd.read_csv(r'C:\Users\Moi\Downloads\train.csv')
data_test = pd.read_csv(r'C:\Users\Moi\Downloads\test.csv')

### Exploration des datasets

In [78]:
# Afficher les premières lignes des jeux de données
data_gender.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [79]:
data_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [80]:
data_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [81]:
# Informations générales sur les datasets
data_gender.info()
data_train.info()
data_test.info()

# Quelques statistiques descriptives
data_train.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   PassengerId  418 non-null    int64
 1   Survived     418 non-null    int64
dtypes: int64(2)
memory usage: 6.7 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtype

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


### Nettoyage des données

In [82]:
# Remplacer les valeurs manquantes de l'age dans les datasets "train" et "test"

data_train['Age'] = data_train['Age'].fillna(data_train['Age'].median())
data_test['Age'] = data_test['Age'].fillna(data_test['Age'].median())


In [83]:
# Remplacer les valeurs manquantes dans 'Fare' pour le dataset de "test" avec la médiane

data_test['Fare'] = data_test['Fare'].fillna(data_test['Fare'].median())

In [84]:
# Remplacer les valeurs manquantes dans 'Embarked' avec la valeur la plus fréquente

most_common_embarked = data_train['Embarked'].mode()[0]
data_train['Embarked'] = data_train['Embarked'].fillna(most_common_embarked)
data_test['Embarked'] = data_test['Embarked'].fillna(most_common_embarked)

In [85]:
# Indication binaire pour `Cabin`
data_train['HasCabin'] = data_train['Cabin'].notnull().astype(int)
data_test['HasCabin'] = data_test['Cabin'].notnull().astype(int)

In [86]:
# Encodage des données de `Sex`
data_train['Sex'] = data_train['Sex'].map({'male': 0, 'female': 1})
data_test['Sex'] = data_test['Sex'].map({'male': 0, 'female': 1})

### Selection des caractéristiques du modèle

In [87]:
# Encodage des variables catégorielles
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'HasCabin']),
        ('cat', OneHotEncoder(), ['Sex', 'Embarked'])
    ])

# Appliquer la transformation
X_train_encoded = preprocessor.fit_transform(X_train)
X_test_encoded = preprocessor.transform(X_test)

In [88]:
# Vérification des colonnes
print(data_train.columns)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'HasCabin'],
      dtype='object')


In [89]:
# Selection des caractéristiques
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'HasCabin']
X_train = data_train[features]
y_train = data_train['Survived']
X_test = data_test[features]

In [90]:
# Entraîner le modèle
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_encoded, y_train)

RandomForestClassifier(random_state=42)

In [91]:
# Prédire sur les données d'entraînement
y_pred_train = model.predict(X_train_encoded)
print(f"Accuracy sur l'entraînement: {accuracy_score(y_train, y_pred_train):.2f}")
print("Matrice de confusion :")
print(confusion_matrix(y_train, y_pred_train))

Accuracy sur l'entraînement: 0.98
Matrice de confusion :
[[547   2]
 [ 13 329]]


In [92]:
# Prédictions sur les données de test
predictions = model.predict(X_test_encoded)

# Préparer le fichier de soumission
submission = pd.DataFrame({
    'PassengerId': data_test['PassengerId'],
    'Survived': predictions
})

# Sauvegarder le fichier de soumission
submission.to_csv('submission.csv', index=False)

submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,1
4,896,0
